In [1]:
from bookimed_lib import *
from math import sqrt

In [ ]:
#ed = [ 1.36042885,  6.88723337, -2.75371749,  6.10902385] #30%
ed = [ 11.91556902,   2.57126826,   2.02539935,   7.31073568] #10% with % no 0
gd = [ 2.20655132,  0.56137078, -0.34676471,  3.05318613,  3.00630209, 9.72031092] #20%
#gp = [ 8.21105664, -33.91364673,  11.01504383,  27.19710437] #50%
gp = [  4.00621326, -16.21028222,   5.83723422,  23.46461818] #20% with %

In [2]:
ed = [ 7.97595949,  2.99958054,  3.14614609,  6.00143112] #30% with % no 0
gd = [ 6.2984266 , -1.61526884,  3.0026383 ,  0.97934334,  0.34856172, 4.35150172] #40%
gp = [  2.38142122,  -9.95300862,   6.09372163,  17.10517242] #20% with %

In [3]:
FEATURES = [
    "gd",
    "ed",
    "gp",
#    "isr",
    "large", 
     "top", 
#    "speed_level",
    "speed_level 1",
    "speed_level 2",
    "speed_level 3",
    "price",
#    "price_level 1",
#    "price_level 2",
#    "price_level 3",
#    "views", 
]

In [4]:
def get_speed_level(speed_level):
    if speed_level == 1:
        #return  [1]
        return [1,0,0]
        #print X[-1], speed_level
    elif speed_level == 2:
        #return [0.4]
        return [0,1,0]
        #print X[-1], speed_level
    elif speed_level == 3:
        #return [0.1]
        return [0,0,1]
        #print X[-1], speed_level
    else:
        #return [0]
        return [0,0,0]
        #print X[-1], speed_level

In [5]:
def extract_data(clinic, clinic_estim):
    all_proc = clinic['procedures']['diagnostics'] + clinic['procedures']['operations']
    all_doc = clinic['doctors']
    X = []
    for doc in all_doc:
        ids_diad = [i['id'] for i in doc['procedures']['diagnostics'] if i['top']=='1']
        ids_oper = [i['id'] for i in doc['procedures']['operations'] if i['top']=='1']
        if doc['procedures']['diagnostics'] != [] and doc['procedures']['operations'] != []:
            X+=[[]]
            #gd 0-6
            X[-1]+=[doc['degree'].count(u'Профессор')]
            X[-1]+=[int(doc['top'])]
            X[-1]+=[len(doc['illnesses'])]
            X[-1]+=[len(doc['languages'])]
            X[-1]+=[int(doc['h_index'])]
            X[-1]+=[int(doc['experience'])]
            #ed 6-10
            #X[-1]+=[len(list(set(ids_diad)))]
            if len(doc['procedures']['diagnostics']) != 0:
                X[-1]+= [sqrt(len(list(set(ids_diad))) / float(len(doc['procedures']['diagnostics'])))]
            else:
                X[-1] += [0]
            X[-1]+=[len(doc['procedures']['diagnostics'])]
            #X[-1]+=[len(list(set(ids_oper)))]
            if len(doc['procedures']['operations']) != 0:
                X[-1] += [sqrt(len(list(set(ids_oper))) / float(len(doc['procedures']['operations'])))]
            else:
                X[-1] += [0]
            X[-1]+=[len(doc['procedures']['operations'])]
            #gp 10-14
            X[-1]+=[sqrt(sum([int(i['top']) for i in clinic['procedures']['diagnostics']]) / float(len(clinic['procedures']['diagnostics'])))] #top_proc_cli
            X[-1]+=[len(clinic['procedures']['diagnostics'])] #gen_proc_cli
            X[-1]+=[sqrt(sum([int(i['top']) for i in clinic['procedures']['operations']]) / float(len(clinic['procedures']['operations'])))] #top_oper_cli
            X[-1]+=[len(clinic['procedures']['operations'])] #gen_oper_cli
            #
            c_id = int(clinic['country_id'])
    #        if c_id == 1:
    #            X[-1]+=[1]
    #        else:
    #            X[-1]+=[0]
            #X[-1]+=[int(clinic['views'])]
            X[-1]+=[int(clinic['large'])]
            X[-1]+=[int(clinic['top'])]
            #X[-1]+=[int(clinic['speed_level'])]
            X[-1]+=get_speed_level(int(clinic['speed_level']))
            #X[-1]+=get_speed_level(int(clinic['price_level']))
            X[-1]+=[round(int(clinic['price_level'])/3.0,4)]

            #X[-1] += [clinic['country_id']]
            #X[-1]+=[int(clinic['count_reviews'])]
            #X[-1]+=[int(clinic["count_orders"])]
        else:
            print clinic['id']
    if clinic_estim != []:
        y=[int(clinic_estim['rating'])] * len(X)
        return [X, y]
    else:
        return X

In [6]:
def pack(X, gd, ed, gp):
    new_X = []
    for x in X:
        new_X+= [sum([
            [round(sum(np.array(gd)*np.array(x[0:6])),5)],
            [round(sum(np.array(ed)*np.array(x[6:10])),5)],
            [round(sum(np.array(gp)*np.array(x[10:14])),5)],
            x[14:]
        ],[])]
    return new_X

In [7]:
#1
def normale(mat):
    for i in range(0,len(mat[0])):
        mass = [abs(k[i]) for k in mat]
        m = max(mass)
        if m>0:
            for j in mat:
                j[i] = round(j[i] / float(m), 4)

In [8]:
#2
def normale(mat):
    for i in range(0,len(mat[0])):
        mass = [k[i] for k in mat]
        mean = np.mean(mass)
        mx = max(mass)
        mn = min(mass)
        if mx!=mn:
            for j in mat:
                #print j[i], j[i] - mn, mx-mn
                j[i] = round((j[i] - mn) / float(mx-mn), 4)
                #print j[i]

In [ ]:
#3
def normale(mat):
    for i in range(0,len(mat[0])):
        mass = [k[i] for k in mat]
        mean = np.mean(mass)
        mx = max(mass)
        mn = min(mass)
        if mx!=mn:
            for j in mat:
                j[i] = round((j[i] - mean) / float(mx-mn), 4)

In [ ]:
#4
def normale(mat):
    for i in range(0,len(mat[0])):
        mass = [k[i] for k in mat]
        mean = np.mean(mass)
        if mean!=0:
            for j in mat:
                j[i] = round((j[i] / mean), 4)
        else:
            print mean, mass

In [ ]:
#5
def normale(mat):
    for i in range(0,len(mat[0])):
        mass = [k[i] for k in mat]
        mean = np.mean(mass)
        std = np.std(mass)
        if std!=0:
            for j in mat:
                #print j[i], j[i] - mn, mx-mn
                j[i] = round((j[i] - mean) / std, 4)
                #print j[i]
        else:
            print std, mass

In [9]:
X_all, X_1, X_2, X_3, X_4, X_5, y_all, y_1, y_2, y_3, y_4, y_5 = get_X_sets(extract_data)
normale(X_all)
X_all=pack(X_all,gd, ed, gp)
normale(X_all)

normale(X_1)
X_1=pack(X_1,gd, ed, gp)
normale(X_1)

normale(X_2)
X_2=pack(X_2,gd, ed, gp)
normale(X_2)

normale(X_3)
X_3=pack(X_3,gd, ed, gp)
normale(X_3)

normale(X_4)
X_4=pack(X_4,gd, ed, gp)
normale(X_4)
print
print
normale(X_5)
X_5=pack(X_5,gd, ed, gp)
normale(X_5)

sp = 100

1
1
8
134
8
8
8
1
1
1
1
1
8
8
1
1
8
1
1
8




In [ ]:
X_all, X_1, X_2, X_3, X_4, X_5, y_all, y_1, y_2, y_3, y_4, y_5 = get_X_sets(extract_data)

X_all[0][14:]

In [10]:
len(X_all)

44

In [ ]:
for X in [X_1, X_2, X_3, X_4, X_5]:
    a=pd.ewma(pd.DataFrame([np.linalg.norm(i[0]) for i in X]), span=sp)
    b=pd.ewma(pd.DataFrame([np.linalg.norm(i[1]) for i in X]), span=sp)
    c=pd.ewma(pd.DataFrame([np.linalg.norm(i[2]) for i in X]), span=sp)
    for i in range(0, len(X)):
        X[i][0:3] = [a[0][i], b[0][i], c[0][i]]

In [ ]:
r=0
l=0
a = []
b = []
c = []

for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
    l=r
    r+=length
    a=pd.ewma(pd.DataFrame([np.linalg.norm(i[0]) for i in X_all[l:r]]), span=sp)
    b=pd.ewma(pd.DataFrame([np.linalg.norm(i[1]) for i in X_all[l:r]]), span=sp)
    c=pd.ewma(pd.DataFrame([np.linalg.norm(i[2]) for i in X_all[l:r]]), span=sp)
    for i in range(0, length):
        X_all[l+i][0:3] = [a[0][i], b[0][i], c[0][i]]

In [ ]:
for i in range(1,7):
    print i
    regr = process_with(X_all, y_all, info=True, ts=i/10.0)
    print

In [15]:
regr = process_with(X_all, y_all, info=True, ts=0.1)

Total: 44, train: 39, test: 5
Residual sum of squares: 3.24
Train absolute: 1.76
Test absolute: 1.64
Absolute to mean: 30.45%
Train variance score: 0.49
Test variance score: 0.54


In [ ]:
for x in [X_1, X_2, X_3, X_4, X_5]:
    print "min %f" % min([i[2] for i in x])
    print "max %f" % max([i[2] for i in x])
    print

In [ ]:
for X,y in [(X_1, y_1),(X_2, y_2),(X_3, y_3),(X_4, y_4),(X_5, y_5)]:
    print "---"
    regr = process_with(X, y, ts=get_best_ts(X, y))
    print "---!"
    f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
    for i in f:
        #print "%4f \t %s" %(i[0], i[1])
        print i[0] #, i[1]
    print regr.intercept_
    print "!"
    f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
    ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
    for i in ind:
        print i+1

In [ ]:
print "Features sorted by their score:"
#f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
print len(FEATURES)
for i in f:
    print "%4f \t %s" %(i[0], i[1])
    #print i[0] #, i[1]
print regr.intercept_

In [ ]:
f = sorted(zip(map(lambda x: round(x, 4), regr.coef_), FEATURES), reverse=True)
ind = [map(lambda x: x[1], f).index(i) for i in FEATURES]
for i in ind:
    print i+1

In [ ]:
regr = process_with(X_1, y_1, ts=get_best_ts(X_1, y_1))

In [ ]:
this_path

In [ ]:
with open('%s/меланома_все.json'%this_path) as data_file: #2
#with open('%s/рак_груди_все.json'%this_path) as data_file: #0
#with open('%s/рак_простаты_все.json'%this_path) as data_file: #4
#with open('%s/рак_шейки_матки_все.json'%this_path) as data_file: #1
#with open('%s/рак_щитовидки_все.json'%this_path) as data_file: #3
    cancer_data = json.load(data_file)[0]['clinics']
X_test, names, ids = get_X_from(cancer_data, extract_data)
normale(X_test)
X_test = pack(X_test,gd, ed, gp)
normale(X_test)

In [ ]:
import warnings
warnings.filterwarnings("ignore")
results=[]
for i in range(0, len(X_test)):
    results += [(regr.predict(X_test[i])[0], names[i])]

res=[]
for n in set(names):
    indices = [j for j, x in enumerate(results) if x[1] == n]
    m = max([i[0] for i in results[min(indices):max(indices)+1]])
    res+=[(m,n, ids[indices[0]])]

for r in res:
#for r in sorted(results, reverse=True):
    #print "%2.2f \t %s" % (r[0], r[1])
    print r[0]

In [ ]:
pos = [10 - sorted(res, reverse=True).index(x) for x in res]
for p in pos:
    print p

In [ ]:
import warnings
warnings.filterwarnings("ignore")
for i in range(0,len(y_2)):    
    print "Real: %f \t Predicted: %f" %(y_2[i], regr.predict(X_2[i]))
    print X_2[i]
    print
    #print y_all[i]
    #print regr.predict(X_all[i])[0]

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    k=0
    for X in [X_1, X_2, X_3, X_4, X_5]:
        k += 1
        plt.plot(sorted([i[f] for i in X]))
        #plt.show()
        plt.savefig('%s - X%d.png' % (feature, k))
        plt.clf()
    f+=1

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    mass = [i[f] for i in X_all]
    k=0
    r=0
    l=0
    for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
        l=r
        r+=length
        k+=1
        plt.plot(sorted(mass[l:r]), label="X%d"%k)
        plt.ylim([min(mass),1])
        legend = plt.legend(loc=4, shadow=True)
        #plt.show()
    plt.savefig('%s.png' % (feature))
    plt.clf()
    f+=1

In [ ]:
plt.plot([np.linalg.norm(i) for i in X_5], color='black')
plt.xticks(())
plt.yticks(())
plt.show()

In [ ]:
f=0
for feature in ["gd", "ed", "gp"]:
    mass = [i[f] for i in X_all]
    k=0
    r=0
    l=0
    for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
        l=r
        r+=length
        k+=1
        plt.plot(y_all[l:r], [i[f] for i in X_all[l:r]], label="X%d"%k)
        plt.ylim([min(mass),1])
        plt.xlim([0,11])
        legend = plt.legend(loc=4, shadow=True)
        plt.savefig('%s - X%d.png' % (feature, k))
        plt.show()
        plt.clf()
    f+=1

In [ ]:
k=0
r=0
l=0
for length in [len(X_1), len(X_2), len(X_3), len(X_4), len(X_5)]:
    l=r
    r+=length
    k+=1
    plt.plot(y_all[l:r], [np.linalg.norm(i) for i in X_all[l:r]], label="X%d"%k)
    legend = plt.legend(loc=4, shadow=True)
    plt.xlim([0,11])
    plt.savefig('norms - X%d.png'%k)
    plt.clf()

In [ ]:
print FEATURES[2]
print [i[0] for i in X_1]
print [i for i in y_1]

In [ ]:
for k in range(0,26):
    print FEATURES[k], 10 - [i[k] for i in X].count(0)

In [ ]:
x = [i[-2] for i in X_all]
y_top = []
y_n_top = []
for l in range(0, len(x)):
    if x:
        y_top += [y[l]]
    else:
        y_n_top += [y[l]]
#for l in range(0, len(x)):
    #print x[l], y[l]

In [ ]:
for i in range(1,11):
    print i, y_top.count(i)

In [ ]:
plt.plot( x, y_all, 'ro')
plt.show()

In [ ]:
#for table filling
res_X = []
res_y = []
for X,y in [(X_1, y_1),(X_2, y_2),(X_3, y_3),(X_4, y_4),(X_5, y_5)]:
#for X,y in [(X_5, y_5)]:
    for i in range(10,0,-1):
        try:
            indices = [j for j, x in enumerate(y) if x == i]
            res_X += [max(regr.predict(X[min(indices):max(indices)+1]))]
            res_y += [i]
            #print res_X[-1], res_y[-1]
        except Exception, err:
            #print err, i, indices
            pass
for x in res_X:
    print x

In [ ]:
pl = [ round(3*i[-1],0) for i in X_all]
plt.hist(pl, bins=4)
plt.title("Price level distribution")
plt.xlabel("Price level")
plt.ylabel("Frequency")

plt.show()

In [ ]:
indices = [i for i, x in enumerate(y_1) if x == 2]
print indices
for ind in indices:
    print regr.predict(X_1[ind])

print max(regr.predict(X_1[min(indices):max(indices)+1]))

In [ ]:
import xlsxwriter

workbook = xlsxwriter.Workbook('Positions.xlsx')
#list 1
worksheet = workbook.add_worksheet("Рак кожи")
worksheet.freeze_panes(1, 0)
col = 0
row = 0

temp = [u"Клиника", u"Оценка (все)", u"Позиция (все)", u"Оценка (свои)", u"Позиция (свои)", u"Оценка (свои+vec pl)", u"Позиция (свои+vec pl)", u"Оценка врача", u"Позиция врача", u"Позиция на сайте"]
for i in temp:
    worksheet.write(row, col, i)
    col+=1

row=1
col=0

# num and clinic name for any model
for i, el in enumerate(zip(res,pos)):
    worksheet.write(row, 0, i)
    print el[0][1]
    worksheet.write_string(row, 1, el[0][1].encode('utf-8').replace('ё', 'е'))
    row+=1

#rate and pos of model
for el in zip(res,pos):    
    worksheet.write(row, 2, el[0][0])
    worksheet.write(row, 3, el[1])

workbook.close()

In [ ]:
len(X_all)

In [16]:
#creare xlsx file for model trained on ALL data

import xlsxwriter

#pre
f = zip(map(lambda x: round(x, 4), regr.coef_), FEATURES)
#f = zip(map(lambda x: round(x, 4), [0] * len(FEATURES)), FEATURES)
#for table filling
res_X = []
res_y = []
t_amounts = [0]
amounts = []

for X,y in [(X_1, y_1),(X_2, y_2),(X_3, y_3),(X_4, y_4),(X_5, y_5)]:
    #regr = process_with(X, y, ts=get_best_ts(X, y))
    for i in range(10,0,-1):
        try:
            indices = [j for j, x in enumerate(y) if x == i]
            res_X += [max(regr.predict(X[min(indices):max(indices)+1]))]
            res_y += [i]
            #print res_X[-1], res_y[-1]
        except Exception, err:
            #print err, i, indices
            pass
    t_amounts += [len(res_y)]


for i in range(1, len(t_amounts)):
    amounts += [t_amounts[i] - t_amounts[i-1]]
    
l=0
r=0
all_pos_model = []
all_pos_doctor = []
seq_pr = []
top_3 = []
top_3_strict = []
for i in amounts:
    l=r
    r+=i
    pos_for_x = [10-sorted(res_X[l:r], reverse=True).index(x) for x in res_X[l:r]]
    all_pos_model += pos_for_x
    all_pos_doctor += [10-sorted(res_y[l:r], reverse=True).index(x) for x in res_y[l:r]]
    seq_pr += [seq_procent(pos_for_x)]
    top_3 += [len(set([10,9,8]).intersection(pos_for_x[0:3]))]
    top_3_strict += [sum(pos_for_x[0:3] == np.array([10,9,8]))]

ranges = []
r=1
for a in amounts:
    l=r+1
    r=l+a-1
    ranges+= [(l,r)]


illnesses = [u"Рак кожи", u"Рак груди", u"Рак простаты", u"Рак шейки матки", u"Рак щитовидки"]


workbook = xlsxwriter.Workbook('./reports/to_choose/M17:old data, sp, pl, om intercept,sqrt % insted top, no 0 and, 10%.xls')

#list 1
worksheet = workbook.add_worksheet("coef")
worksheet.write(0, 0, "f")
worksheet.write(0, 2, "rating")

row = 1
col = 2

for num, feat in f:
    worksheet.write(row, col, feat)
    worksheet.write(row, col+1, num)
    row+=1

worksheet.write(row, col, "intercept")
worksheet.write(row, col+1, regr.intercept_)
row+=2
col=0

r=row

worksheet.write(row, col, "GD")
worksheet.write(row, col+2, "specialist")
row+=1

f_ = zip(map(lambda x: round(x, 4), gd), ["deg", "top", "illn", "lang", "h_index", "exp", "intercept"])
for i, el in enumerate(f_):
    worksheet.write(row, col, "c_%d"%i)
    worksheet.write(row, col+1, el[0])
    worksheet.write(row, col+2, el[1])
    row+=1

row = r
col += 4
worksheet.write(row, col, "ED")
worksheet.write(row, col+2, "experience")
row+=1

f_ = zip(map(lambda x: round(x, 4), ed), ["top_diag", "gen_diag", "top_oper", "gen_oper", "intercept"])
for i, el in enumerate(f_):
    worksheet.write(row, col, "c_%d"%i)
    worksheet.write(row, col+1, el[0])
    worksheet.write(row, col+2, el[1])
    row+=1
    

row = r
col += 4
worksheet.write(row, col, "GP")
worksheet.write(row, col+2, "experience")
row+=1

f_ = zip(map(lambda x: round(x, 4), gp), ["top_diag", "gen_diag", "top_oper", "gen_oper"])
for i, el in enumerate(f_):
    worksheet.write(row, col, "c_%d"%i)
    worksheet.write(row, col+1, el[0])
    worksheet.write(row, col+2, el[1])
    row+=1

    
#list 2
worksheet = workbook.add_worksheet("tabl")
worksheet.freeze_panes(1, 0)
col = 0
row = 0

for i in [u"Болезнь", u"Клиника", u"Оценка врача", u"Оценка модели", u"Позиция врача", u"Позиция модели", u"Разница оценок врача и модели", u"Разница позиций врача и модели"]:
    worksheet.write(row, col, i)
    col+=1

col=0
row=1
temp = sum([[i[0]]*i[1] for i in zip(illnesses, amounts)],[])
for i in temp:
    worksheet.write(row, col, i)
    row+=1

col=1
row=1
temp = get_clinic_names_all()
for i in temp:
    worksheet.write(row, col, i)
    row+=1
    
col=2
row=1
for i in res_y:
    worksheet.write(row, col, i)
    row+=1

col=3
row=1
for i in res_X:
    worksheet.write(row, col, i)
    row+=1
    
col=4
row=1
for i in all_pos_doctor:
    worksheet.write(row, col, i)
    row+=1

col=5
row=1
for i in all_pos_model:
    worksheet.write(row, col, i)
    row+=1


col=6
row=1
for i in range(0, sum(amounts)):
    worksheet.write_formula(row, col, '=C%d-D%d'%(row+1,row+1))
    worksheet.write_formula(row, col+1, '=E%d-F%d'%(row+1,row+1))
    worksheet.write_formula(row, col+2, '=ABS(G%d)'%(row+1))
    worksheet.write_formula(row, col+3, '=ABS(H%d)'%(row+1))
    row+=1

for l,r in ranges:
    worksheet.write_formula('K%d' % r, '=AVERAGE(I%d:I%d)'%(l,r))
    worksheet.write_formula('L%d' % r, '=AVERAGE(J%d:J%d)'%(l,r))

#list 3
worksheet = workbook.add_worksheet("stat")
worksheet.freeze_panes(1, 0)
col = 1
row = 0

for i in [ u"Среднее по оценке", u"Среднее по позиции", u"% последовательности", u"топ-3 с 10", u"топ-3"]:
    worksheet.write(row, col, i)
    col+=1

col=0
row=1
for i in illnesses + ['', u"Итого"]:
    worksheet.write(row, col, i)
    row+=1

col=1    
row=1
for i in [i[1] for i in ranges]:
    worksheet.write(row, col, "='tabl'!K%d"%(i))
    worksheet.write(row, col+1, "='tabl'!L%d"%(i))
    row+=1

col=3
row=1
for i in zip(seq_pr,top_3_strict, top_3):
    worksheet.write(row, col, i[0])
    worksheet.write(row, col+1, i[1])
    worksheet.write(row, col+2, i[2])
    row+=1
    
row+=1
col=1
for i in ['B', 'C', 'D', 'E', 'F']:
    worksheet.write_formula(row, col, '=AVERAGE(%s2:%s6)'%(i,i))
    col+=1

workbook.close()